# Project 3 - Spatial Analyis and Spatial Auto-correlation
### Lorn Jaeger


Get the list of hdf5 files

In [ ]:
from pathlib import Path

dpath = Path("/run/host/run/data_raid5/scratch/wsts_temp/")

files = []

for dir in sorted(dpath.iterdir()):
    if dir.is_dir():
        for file in sorted(dir.iterdir()):
            if file.is_file():
                files.append(file)

print(f"{len(files)} fires")

Turn hdf5 files to tabular data. 

In [2]:
import h5py
import numpy as np
import pandas as pd

feature_names = [
    'M11', 'I2', 'I1', 'NDVI', 'EVI2',
    'precip', 'wind_speed', 'wind_dir',
    'temp_min', 'temp_max', 'ERC',
    'specific_humidity', 'slope', 'aspect',
    'elevation', 'PDSI', 'landcover',
    'forecast_precip', 'forecast_wind_speed',
    'forecast_wind_dir', 'forecast_temp',
    'forecast_specific_humidity',
    'active_fire_count'
]

rows = []

for file in files:
    with h5py.File(file, "r") as r:
        data = r["data"]                      
        dates = r["data"].attrs["img_dates"]   
        fire_id = r["data"].attrs["fire_name"]
        lon, lat = r["data"].attrs["lnglat"]     

        for i, date in enumerate(dates):
            raster = data[i]                   
    
            b23 = raster[22]
            mask = b23 != 0
            num_active = int(mask.sum())

            if num_active == 0:
                full_mask = np.ones_like(mask, dtype=bool)
            else:
                full_mask = mask

            masked_vals = raster[:22][:, full_mask]   
            means = masked_vals.mean(axis=1)

            lc_vals = raster[16][full_mask].astype(int, copy=False)
            majority_lc = np.bincount(lc_vals).argmax()
            means[16] = majority_lc

            row = list(means) + [num_active]

            rows.append({
                "fire_id": fire_id,
                "date": str(date),
                "lat": lat,
                "lon": lon,
                **{feature_names[j]: row[j] for j in range(23)}
            })

df = pd.DataFrame(rows)
df
df.to_csv("output.csv", index=False)

Fill in NaN values

In [3]:
df_filled = df.copy()

num_cols = df_filled.select_dtypes(include=[np.number]).columns

global_means = df_filled[num_cols].mean()

for col in num_cols:
    fire_means = df_filled.groupby("fire_id")[col].transform("mean")
    
    fallback = global_means[col]
    col_fill = fire_means.fillna(fallback)
    
    df_filled[col] = df_filled[col].fillna(col_fill)

Add in the next day fires.

In [ ]:
df = df_filled.copy()
df = df.sort_values(["fire_id", "date"])
df["next_day_fire"] = df.groupby("fire_id")["active_fire_count"].shift(-1)
df = df.dropna(subset=["next_day_fire"])
df["next_day_fire"] = df["next_day_fire"].astype(float)

At this point we have our final list of fires. Now we can merge our other data.

In [10]:
%reset -f

import pyogrio
import pandas as pd

df_filled = pd.read_csv("../filled_output.csv")


df = pyogrio.read_dataframe(
    "../data/output.gpkg",
    layer="US_WUI_block_1990_2020_change_v4",
    columns=["WUIFLAG2020"]
)

print(df["WUIFLAG2020"].unique())


KeyboardInterrupt: 